# Task 3

The provided UNetCNN.py contains a U-Net architecture convolutional neural network, which was originally used for biomedical image segmenation. 

Our goal is not to necessarily fully understand the reason behind the implementation, but it will be to explain and analyze the features of the network that have thus far been analyzed. Using the techniques from the analysis in the notes and the walkthrough of the creation of the CNN in Keras, answer the following questions.

##### (a) Draw a diagram of the convolutional neural network provided in the code. Note that this model uses the functional model, which essentially behaves similarly to a linked list of layers. Refer to the note for examples on how to implement and reason with these models.

#####  (b) Discuss a potential reason why the number of filters stays the same with each convolution in the first half of the model. Why do we implement pooling in the model?

(YOUR ANSWER HERE)

#####  (c) What could be a side effect of pooling too much in this model?

(YOUR ANSWER HERE)

#####  (d) Find the depth of this model. Consider one convolutional layer as one layer as opposed to the combination of convolutional layers and pooling layers as one layer.

(YOUR ANSWER HERE)

#####  (e) What purpose could the calls to concatenate serve? What kinds of layers are we concatenating with each other?

(YOUR ANSWER HERE)

##### (f) What kind of activation function is most prevalent? Is this typical or atypical of a convolutional neural network?

(YOUR ANSWER HERE)

##### (g) Discuss why this Keras model utilizes a Dropout layer and what potential side effects may be.

(YOUR ANSWER HERE)

##### (h) Why do we use a sigmoid activation function near the end of the network? How is this related to the normalization technique that is employed?

(YOUR ANSWER HERE)

##### (i) Discuss potential improvements to the model or any red flags.

(YOUR ANSWER HERE)

##### (j) Run the model. What is the test accuracy? Why does this make sense?

(YOUR ANSWER HERE)

In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, UpSampling2D, concatenate, Dropout, Cropping2D

(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.mnist.load_data(path="mnist.npz")

X_train = X_train / 255
X_test = X_test / 255

# one-hot encode to make this a classification task 
Y_train = keras.utils.to_categorical(Y_train, 10)
Y_test = keras.utils.to_categorical(Y_test, 10)

# (x, y, #channels)
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)


def get_crop_shape(target, refer):
    """From StackExchange question"""
    # width, the 3rd dimension
    cw = (target.get_shape()[2] - refer.get_shape()[2])
    assert (cw >= 0)
    if cw % 2 != 0:
        cw1, cw2 = int(cw/2), int(cw/2) + 1
    else:
        cw1, cw2 = int(cw/2), int(cw/2)
    # height, the 2nd dimension
    ch = (target.get_shape()[1] - refer.get_shape()[1])
    assert (ch >= 0)
    if ch % 2 != 0:
        ch1, ch2 = int(ch/2), int(ch/2) + 1
    else:
        ch1, ch2 = int(ch/2), int(ch/2)

    return (ch1, ch2), (cw1, cw2)

# downwards
inp = keras.Input(shape = (28, 28, 1))
c1 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(inp)
c1 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(c1)
# can comment next line and uncomment the following line for uncompressed
inter = MaxPooling2D((2, 2))(c1)
# inter = c1
c2 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(inter)
c2 = Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(c2)
# can comment next line and uncomment the following line for uncompressed
inter = MaxPooling2D((2, 2))(inter)
inter = Dropout(0.5)(inter)
# inter = c2
inter = Conv2D(32, (3, 3), activation = 'relu')(inter)
inter = Conv2D(32, (3, 3), activation = 'relu')(inter)

# upwards
inter = UpSampling2D((2, 2))(inter)
c2_crop = Cropping2D(cropping=get_crop_shape(c2, inter)[0], data_format = "channels_last")(c2)
inter = concatenate([inter, c2_crop])
inter = Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(inter)
inter = Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(inter)
inter = UpSampling2D((2, 2))(inter)
c1_crop = Cropping2D(cropping=get_crop_shape(c1, inter)[0], data_format = "channels_last")(c1)
inter = concatenate([inter, c1_crop])
inter = Conv2D(32, (3, 3), activation = 'relu')(inter)
inter = Conv2D(32, (3, 3), activation = 'relu')(inter)
inter = Conv2D(1, (1, 1), activation = 'sigmoid')(inter)
out = Flatten()(inter)
# softmax outperformed every other activation fn here
out = Dense(32, activation = 'relu')(out)
out = Dense(10, activation = 'softmax')(out)

# softmax + binary CE > sigmoid + binary CE >>>>> anything + categ CE
model = keras.Model(inputs=inp, outputs=out, name="mnist_model")
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(X_train, Y_train, batch_size = 32, epochs = 12, validation_split = 0.2)

print(model.evaluate(X_test, Y_test, verbose=1))